In [ ]:
import pandas as pd
import numpy as np
import nltk
nltk.download('wordnet')
import re
from bs4 import BeautifulSoup

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import Perceptron, LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC, LinearSVC
import gensim.downloader as api
import pandas as pd
import numpy as np
from gensim.utils import simple_preprocess
from gensim.models import Word2Vec


import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
import torch.nn.functional as F


[nltk_data] Downloading package wordnet to C:\Users\Aditya
[nltk_data]     Roy\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Read Data

In [ ]:
#Absolute Path to dataset

data = pd.read_csv(r'C:\Users\Aditya Roy\Downloads\Data\Dataset.tsv', sep=r"\t")

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3419564228.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  data = pd.read_csv(r'C:\Users\Aditya Roy\Downloads\Data\Dataset.tsv', sep=r"\t")


In [ ]:
data.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,43081963,R18RVCKGH1SSI9,B001BM2MAC,307809868,"Scotch Cushion Wrap 7961, 12 Inches x 100 Feet",Office Products,5,0,0,N,Y,Five Stars,Great product.,2015-08-31
1,US,10951564,R3L4L6LW1PUOFY,B00DZYEXPQ,75004341,"Dust-Off Compressed Gas Duster, Pack of 4",Office Products,5,0,1,N,Y,"Phffffffft, Phfffffft. Lots of air, and it's C...",What's to say about this commodity item except...,2015-08-31
2,US,21143145,R2J8AWXWTDX2TF,B00RTMUHDW,529689027,Amram Tagger Standard Tag Attaching Tagging Gu...,Office Products,5,0,0,N,Y,but I am sure I will like it.,"Haven't used yet, but I am sure I will like it.",2015-08-31
3,US,52782374,R1PR37BR7G3M6A,B00D7H8XB6,868449945,AmazonBasics 12-Sheet High-Security Micro-Cut ...,Office Products,1,2,3,N,Y,and the shredder was dirty and the bin was par...,Although this was labeled as &#34;new&#34; the...,2015-08-31
4,US,24045652,R3BDDDZMZBZDPU,B001XCWP34,33521401,"Derwent Colored Pencils, Inktense Ink Pencils,...",Office Products,4,0,0,N,Y,Four Stars,Gorgeous colors and easy to use,2015-08-31


## Keep Reviews and Ratings

For now ['star_rating','review_headline','review_body'] has been loaded. 'review_headline' has been added with 'review_body' and the whole text is used for sentiment classification and 'review_headline' is dropped.  
Adding the headline with the body increases the accuracy by 2%.

In [ ]:
dataset = data[['star_rating', 'review_body']]

dataset["review_body"] = data["review_headline"] + ": " + data["review_body"]
dataset['review_body'] = dataset['review_body'].apply(str)

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3849179413.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["review_body"] = data["review_headline"] + ": " + data["review_body"]
C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3849179413.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['review_body'] = dataset['review_body'].apply(str)


1 - Positive  
0 - Negative  
2 - Neutral

In [ ]:
def createLabel(r):
    if r>3:
        return 1
    if r == 3:   #neutral
        return 2
    else:
        return 0

dataset['labels'] = dataset['star_rating'].apply(createLabel)


C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\4046678660.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset['labels'] = dataset['star_rating'].apply(createLabel)


In [ ]:
# Drop NaN values
dataset = dataset.dropna(subset=['review_body']).reset_index(drop=True)
dataset = dataset.dropna(subset=['star_rating']).reset_index(drop=True)

Random state specified to facilitate easy replication of results. Random state = 42 has been used throughout this notebook for same reason.

In [ ]:

R1 = dataset[dataset['star_rating']==1].sample(50000, random_state=42)
R2 = dataset[dataset['star_rating']==2].sample(50000, random_state=42)
R3 = dataset[dataset['star_rating']==3].sample(50000, random_state=42)
R4 = dataset[dataset['star_rating']==4].sample(50000, random_state=42)
R5 = dataset[dataset['star_rating']==5].sample(50000, random_state=42)

# BinDataset = pd.concat([R1,R2,R4,R5], ignore_index=True)
ter_dataset = pd.concat([R1,R2,R3,R4,R5], ignore_index=True)

# Data Cleaning



In [ ]:
# Apply gensim's text preprocessing
ter_dataset["tokens"] = ter_dataset["review_body"].apply(lambda x: simple_preprocess(x))

In [ ]:
import gc
del R1,R2,R3,R4,R5, dataset, data
gc.collect()

16

# Pretrained Word2Vec

In [ ]:
word2vec = api.load("word2vec-google-news-300")

def get_average_word2vec(tokens, model, vector_size=300):
    vectors = [np.float32(model[word]) for word in tokens if word in model]
    return np.float32(np.mean(vectors, axis=0)) if vectors else np.float32(np.zeros(vector_size))

def get_10_word2vec(tokens, model, vector_size=300):

    vectors = [np.float32(model[word]) for word in tokens if word in model][:10]

    while len(vectors) < 10:
        vectors.append(np.float32(np.zeros(vector_size)))

    return np.concatenate(vectors, axis=0)


# FOR CNN
def get_average_50_word2vec(tokens, model, vector_size=300):
    vectors = [np.float32(model[word]) for word in tokens if word in model][:50]
    return np.float32(np.mean(vectors, axis=0)) if vectors else np.float32(np.zeros(vector_size))

ter_dataset["pretrained_average_embeddings"] = ter_dataset["tokens"].apply(lambda x: get_average_word2vec(x, word2vec))
ter_dataset["pretrained_average_50_embeddings"] = ter_dataset["tokens"].apply(lambda x: get_average_50_word2vec(x, word2vec))
ter_dataset["pretrained_top_embeddings"] = ter_dataset["tokens"].apply(lambda x: get_10_word2vec(x, word2vec))

# Custom Word2Vec

In [ ]:
custom_word2vec = Word2Vec(sentences=ter_dataset["tokens"], vector_size=300, window=5, min_count=1, workers=4)

def get_average_word2vec(tokens, model, vector_size=300):
    vectors = [np.float32(model.wv[word]) for word in tokens if word in model.wv]
    return np.float32(np.mean(vectors, axis=0)) if vectors else np.float32(np.zeros(vector_size))

def get_10_word2vec(tokens, model, vector_size=300):

    vectors = [np.float32(model.wv[word]) for word in tokens if word in model.wv][:10]

    while len(vectors) < 10:
        vectors.append(np.float32(np.zeros(vector_size)))

    return np.concatenate(vectors, axis=0)


# FOR CNN
def get_average_50_word2vec(tokens, model, vector_size=300):
    vectors = [np.float32(model[word]) for word in tokens if word in model][:50]
    return np.float32(np.mean(vectors, axis=0)) if vectors else np.float32(np.zeros(vector_size))

ter_dataset["custom_average_embeddings"] = ter_dataset["tokens"].apply(lambda x: get_average_word2vec(x, custom_word2vec))
ter_dataset["custom_average_50_embeddings"] = ter_dataset["tokens"].apply(lambda x: get_average_50_word2vec(x, word2vec))
ter_dataset["custom_top_embeddings"] = ter_dataset["tokens"].apply(lambda x: get_10_word2vec(x, custom_word2vec))

In [ ]:
out = word2vec.most_similar(positive=["male","woman"],negative=["man"], topn=1)
print(f"most similar to Male - Man + Woman = {out}")
print(f"Similarity between Outstanding and Excelent = {word2vec.similarity('outstanding','excellent')}")

most similar to Male - Man + Woman = [('female', 0.8426273465156555)]
Similarity between Outstanding and Excelent = 0.5567485690116882


In [ ]:
out = custom_word2vec.wv.most_similar(positive=["male","woman"],negative=["man"],topn=1)
print(f"most similar to Male - Man + Woman = {out}")
print(f"Similarity between Outstanding and Excelent = {custom_word2vec.wv.similarity('outstanding','excellent')}")

most similar to Male - Man + Woman = [('female', 0.7385611534118652)]
Similarity between Outstanding and Excelent = 0.8139455318450928


The pretrained embeddings demonstrate stronger performance in capturing fundamental patterns, reflecting their training on general-purpose text. Meanwhile, the custom embeddings show superior sensitivity to domain-specific words, evidenced by the significantly higher similarity between Outstanding and Excellent. This suggests the custom model better reflects contextual relationships in its training data, while the pretrained model preserves more robust grammatical/semantic regularities. The contrast highlights the trade-off between general linguistic knowledge (pretrained) and task/domain-specific optimization (custom).

#### Creating Binary Dataset by dropping the neutral label (2 or star_rating == 3)

In [ ]:
bin_dataset = ter_dataset[ter_dataset['star_rating']!=3]

# Pre-processing

## remove the stop words

In [ ]:
from nltk.corpus import stopwords
# nltk.download('punkt_tab')
# nltk.download('stopwords')

def removeStopwords(review):
    tokens = nltk.tokenize.word_tokenize(review)
    filtered_tokens = [word for word in tokens if word not in stopwords.words('english')]
    return ' '.join(filtered_tokens)



dataset['review_body'] = dataset['review_body'].apply(removeStopwords)

## perform lemmatization  

In [ ]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

def lemmatize_text(review):
    tokens = nltk.tokenize.word_tokenize(review)
    lemmatized_tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_tokens)

dataset['review_body'] = dataset['review_body'].apply(lemmatize_text)


In [ ]:
#SPlitting

X_trainB, X_testB, y_trainB, y_testB = train_test_split(bin_dataset[['pretrained_top_embeddings', 'pretrained_average_embeddings', 'custom_average_embeddings', 'custom_top_embeddings','pretrained_average_50_embeddings','custom_average_50_embeddings']], bin_dataset['labels'], test_size=0.2, random_state=42)
X_trainT, X_testT, y_trainT, y_testT = train_test_split(ter_dataset[['pretrained_top_embeddings', 'pretrained_average_embeddings', 'custom_average_embeddings', 'custom_top_embeddings','pretrained_average_50_embeddings','custom_average_50_embeddings']], ter_dataset['labels'], test_size=0.2, random_state=42)


In [ ]:

del bin_dataset, ter_dataset
gc.collect()

516

Following function was created to print the metrics as per the specified format.

In [ ]:
def evaluate(y_true, y_pred, dataset_name = None):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)

    print(f"{dataset_name} Metrics:")
    print("accuracy, precision, recall, f1")
    print("{:.4f}".format(accuracy), "{:.4f}".format(precision), "{:.4f}".format(recall), "{:.4f}".format(f1))


#### Multi class classification
def evaluate_multi(y_true, y_pred, dataset_name = None):
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred, average='macro')
    recall = recall_score(y_true, y_pred, average='macro')
    f1 = f1_score(y_true, y_pred, average='macro')

    print(f"{dataset_name} Metrics:")
    print("accuracy, precision, recall, f1")
    print("{:.4f}".format(accuracy), "{:.4f}".format(precision), "{:.4f}".format(recall), "{:.4f}".format(f1))

# Simple Models

## Perceptron

Perceptron was trained and tested for different max_iter values. It was found that in every case the model gives optimal output with just 100 iterations. Hence, 100 was chosen as the final number of iterations.

### Word2Vec features

#### Custom Word2Vec model features

In [ ]:
perceptron_custom = Perceptron(max_iter=100, tol=1e-3, random_state=42)
perceptron_custom.fit(np.stack(X_trainB['custom_average_embeddings'].values), y_trainB.values)

Perceptron(max_iter=100, random_state=42)

In [ ]:
y_test_pred = perceptron_custom.predict(np.stack(X_testB['custom_average_embeddings'].values))

In [ ]:
evaluate(y_testB.values, y_test_pred, "Perceptron on Custom embeddings")

Perceptron on Custom embeddings Metrics:
accuracy, precision, recall, f1
0.8506 0.9363 0.7526 0.8344


#### Pretrained Word2Vec model features

In [ ]:
perceptron_pretrained = Perceptron(max_iter=100, tol=1e-3, random_state=42)
perceptron_pretrained.fit(np.stack(X_trainB['pretrained_average_embeddings'].values), y_trainB.values)

Perceptron(max_iter=100, random_state=42)

In [ ]:
y_test_pred = perceptron_pretrained.predict(np.stack(X_testB['pretrained_average_embeddings'].values))

In [ ]:
evaluate(y_testB.values, y_test_pred, "Perceptron on pretrained embeddings")

Perceptron on pretrained embeddings Metrics:
accuracy, precision, recall, f1
0.8458 0.8071 0.9089 0.8550


## SVM

## Word2Vec

### Custom Word2Vec Features

In [ ]:
SVM_custom = LinearSVC(random_state=42)
SVM_custom.fit(np.stack(X_trainB['custom_average_embeddings'].values), y_trainB.values)

c:\Users\Aditya Roy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\svm\_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [ ]:
y_test_pred = SVM_custom.predict(np.stack(X_testB['custom_average_embeddings'].values))

In [ ]:
y_trainB.values.shape

(160000,)

In [ ]:
evaluate(y_testB.values, y_test_pred, "SVM on Custom embeddings:")

SVM on Custom embeddings: Metrics:
accuracy, precision, recall, f1
0.8985 0.9104 0.8840 0.8970


### Pretrained Word2Vec Features

In [ ]:
SVM_pretrained = LinearSVC(random_state=42)
SVM_pretrained.fit(np.stack(X_trainB['pretrained_average_embeddings'].values), y_trainB.values)

LinearSVC(random_state=42)

In [ ]:
y_test_pred = SVM_pretrained.predict(np.stack(X_testB['pretrained_average_embeddings'].values))

In [ ]:
evaluate(y_testB.values, y_test_pred, "SVM on pretrained embeddings")

SVM on pretrained embeddings Metrics:
accuracy, precision, recall, f1
0.8657 0.8836 0.8425 0.8625


## Simple model comparison

In [ ]:
acc = {
    "model_name":["Perceptron","SVM"],
    "Pretrained_embeddings":[0.8458,0.8657],
    "Custom_embeddings":[0.8548,0.8985],
    "TF-IDF":[0.8871,0.9269]
}

acc = pd.DataFrame(acc)
acc

,model_name,Pretrained_embeddings,Custom_embeddings,TF-IDF
0,Perceptron,0.8458,0.8548,0.8871
1,SVM,0.8657,0.8985,0.9269


The scores reveal three key trends:

TF-IDF Dominance: Both classifiers achieve their highest performance with TF-IDF features, suggesting traditional bag-of-words approaches remain competitive for this specific task, due to strong keyword signals in the data.

Custom Embedding Advantage: Custom embeddings outperform pretrained ones for both models, indicating domain-specific training improves feature relevance despite lower generalization.

Model Capacity: The SVM consistently outperforms the Perceptron across all feature types, highlighting its superior ability to handle complex decision boundaries with all input representations.

# Feedforward Neural Network

In [ ]:
class EmbeddingDataset(Dataset):
    def __init__(self, X_df, y_df):

        self.embeddings = torch.tensor(np.stack(X_df.values), dtype=torch.float32)

        self.labels = torch.tensor(y_df.values, dtype=torch.long)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.embeddings[idx], self.labels[idx]

# Binary Classification MLP
class BinaryMLP(nn.Module):
    def __init__(self, input_dim, hidden1=50, hidden2=10, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden2, 1)
        )

    def forward(self, x):
        return self.net(x).squeeze()

# Ternary Classification MLP
class TernaryMLP(nn.Module):
    def __init__(self, input_dim, hidden1=50, hidden2=10, dropout=0.5):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden1),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden1, hidden2),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden2, 3)
        )

    def forward(self, x):
        return self.net(x)

In [ ]:
# for internal evaluations
def evaluate_(model, data_loader, num_classes, device='cpu'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    criterion = nn.BCEWithLogitsLoss() if num_classes == 2 else nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            if num_classes == 2:
                loss = criterion(outputs, labels.float())
                preds = (torch.sigmoid(outputs) > 0.5).long()
            else:
                loss = criterion(outputs, labels)
                preds = torch.argmax(outputs, dim=1)

            total_loss += loss.item() * inputs.size(0)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy

In [ ]:
def create_dataloaders(X_df, y_df, test_size=0.2, val_size=0.2, batch_size=64):
    X_train, X_temp, y_train, y_temp = train_test_split(
        X_df, y_df,
        test_size=test_size + val_size,
        stratify=y_df
    )


    test_val_size = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp,
        test_size=test_val_size,
        stratify=y_temp
    )


    train_ds = EmbeddingDataset(X_train, y_train)
    val_ds = EmbeddingDataset(X_val, y_val)
    test_ds = EmbeddingDataset(X_test, y_test)


    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_ds, batch_size=batch_size)
    test_loader = DataLoader(test_ds, batch_size=batch_size)

    return train_loader, val_loader, test_loader

In [ ]:
def get_predictions(model, dataloader, device='cpu'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)

            if isinstance(model, BinaryMLP):  # Binary classification
                preds = (torch.sigmoid(outputs) > 0.5).cpu().numpy()
            else:  # Ternary classification
                preds = torch.argmax(outputs, dim=1).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return np.array(all_labels), np.array(all_preds)

In [ ]:
def train_model(model, train_loader, val_loader, num_classes=2, num_epochs=20, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    if num_classes == 2:
        criterion = nn.BCEWithLogitsLoss()
    else:
        criterion = nn.CrossEntropyLoss()

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    best_val_loss = float('inf')
    patience = 3
    trigger_times = 0

    for epoch in range(num_epochs):
        # Training
        model.train()
        train_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)

            if num_classes == 2:
                loss = criterion(outputs, labels.float())
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)

        # Validation
        val_loss, val_acc = evaluate_(model, val_loader, num_classes, device)

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            trigger_times = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping!")
                break

    model.load_state_dict(torch.load('best_model.pth'))
    return model


## Pretrained Word2Vec model features

## Average of Features

### Binary model

In [ ]:

binary_train_loader, binary_val_loader, binary_test_loader = create_dataloaders(X_trainB['pretrained_average_embeddings'], y_trainB, test_size=0.2, val_size=0.2, batch_size=64)
binary_model = BinaryMLP(input_dim=len(X_trainB['pretrained_average_embeddings'].iloc[0]))
trained_binary_model = train_model(binary_model, binary_train_loader, binary_val_loader, num_classes=2)

Epoch 1/20
Train Loss: 0.4374 | Val Loss: 0.3430 | Val Acc: 0.8552
Epoch 2/20
Train Loss: 0.3730 | Val Loss: 0.3245 | Val Acc: 0.8576
Epoch 3/20
Train Loss: 0.3630 | Val Loss: 0.3133 | Val Acc: 0.8655
Epoch 4/20
Train Loss: 0.3559 | Val Loss: 0.3061 | Val Acc: 0.8702
Epoch 5/20
Train Loss: 0.3522 | Val Loss: 0.3010 | Val Acc: 0.8732
Epoch 6/20
Train Loss: 0.3460 | Val Loss: 0.2996 | Val Acc: 0.8740
Epoch 7/20
Train Loss: 0.3423 | Val Loss: 0.2976 | Val Acc: 0.8744
Epoch 8/20
Train Loss: 0.3433 | Val Loss: 0.3007 | Val Acc: 0.8728
Epoch 9/20
Train Loss: 0.3408 | Val Loss: 0.3075 | Val Acc: 0.8640
Epoch 10/20
Train Loss: 0.3379 | Val Loss: 0.2910 | Val Acc: 0.8788
Epoch 11/20
Train Loss: 0.3376 | Val Loss: 0.2935 | Val Acc: 0.8748
Epoch 12/20
Train Loss: 0.3371 | Val Loss: 0.2885 | Val Acc: 0.8778
Epoch 13/20
Train Loss: 0.3347 | Val Loss: 0.2865 | Val Acc: 0.8788
Epoch 14/20
Train Loss: 0.3321 | Val Loss: 0.2852 | Val Acc: 0.8808
Epoch 15/20
Train Loss: 0.3316 | Val Loss: 0.2863 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


Epoch 19/20
Train Loss: 0.3282 | Val Loss: 0.2835 | Val Acc: 0.8809
Early stopping!


In [ ]:
test_true, test_pred = get_predictions(trained_binary_model, binary_test_loader)
evaluate(test_true, test_pred, "Pretrained average features on Binary MLP:")

Pretrained average features on Binary MLP: Metrics:
accuracy, precision, recall, f1
0.8792 0.8896 0.8660 0.8776


### Ternary model

In [ ]:

ternary_train_loader, ternary_val_loader, ternary_test_loader = create_dataloaders(X_trainT['pretrained_average_embeddings'], y_trainT, test_size=0.2, val_size=0.2, batch_size=64)
ternary_model = TernaryMLP(input_dim=len(X_trainT['pretrained_average_embeddings'].iloc[0]))
trained_ternary_model = train_model(ternary_model, ternary_train_loader, ternary_val_loader, num_classes=3)

Epoch 1/20
Train Loss: 0.8451 | Val Loss: 0.7292 | Val Acc: 0.6853
Epoch 2/20
Train Loss: 0.7741 | Val Loss: 0.7215 | Val Acc: 0.6912
Epoch 3/20
Train Loss: 0.7639 | Val Loss: 0.7133 | Val Acc: 0.6952
Epoch 4/20
Train Loss: 0.7591 | Val Loss: 0.6982 | Val Acc: 0.6969
Epoch 5/20
Train Loss: 0.7521 | Val Loss: 0.6988 | Val Acc: 0.6985
Epoch 6/20
Train Loss: 0.7477 | Val Loss: 0.7023 | Val Acc: 0.6935
Epoch 7/20
Train Loss: 0.7453 | Val Loss: 0.6918 | Val Acc: 0.7002
Epoch 8/20
Train Loss: 0.7426 | Val Loss: 0.6871 | Val Acc: 0.7001
Epoch 9/20
Train Loss: 0.7411 | Val Loss: 0.6839 | Val Acc: 0.7016
Epoch 10/20
Train Loss: 0.7391 | Val Loss: 0.6804 | Val Acc: 0.7025
Epoch 11/20
Train Loss: 0.7355 | Val Loss: 0.6825 | Val Acc: 0.7036
Epoch 12/20
Train Loss: 0.7376 | Val Loss: 0.6819 | Val Acc: 0.6993
Epoch 13/20
Train Loss: 0.7355 | Val Loss: 0.6832 | Val Acc: 0.7048
Early stopping!


C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_ternary_model, ternary_test_loader)
evaluate_multi(test_true, test_pred, "Pretrained average features on ternary MLP:")

Pretrained average features on ternary MLP: Metrics:
accuracy, precision, recall, f1
0.7022 0.4685 0.5850 0.5202


c:\Users\Aditya Roy\AppData\Local\Programs\Python\Python38\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## First 10 Features

### Binary Model

In [ ]:

binary_train_loader, binary_val_loader, binary_test_loader = create_dataloaders(X_trainB['pretrained_top_embeddings'], y_trainB, test_size=0.2, val_size=0.2, batch_size=64)
binary_model = BinaryMLP(input_dim=len(X_trainB['pretrained_top_embeddings'].iloc[0]))
trained_binary_model = train_model(binary_model, binary_train_loader, binary_val_loader, num_classes=2)

Epoch 1/20
Train Loss: 0.4072 | Val Loss: 0.3030 | Val Acc: 0.8666
Epoch 2/20
Train Loss: 0.3462 | Val Loss: 0.2956 | Val Acc: 0.8696
Epoch 3/20
Train Loss: 0.3300 | Val Loss: 0.2875 | Val Acc: 0.8742
Epoch 4/20
Train Loss: 0.3166 | Val Loss: 0.2837 | Val Acc: 0.8753
Epoch 5/20
Train Loss: 0.3072 | Val Loss: 0.2810 | Val Acc: 0.8788
Epoch 6/20
Train Loss: 0.2975 | Val Loss: 0.2813 | Val Acc: 0.8785
Epoch 7/20
Train Loss: 0.2893 | Val Loss: 0.2801 | Val Acc: 0.8786
Epoch 8/20
Train Loss: 0.2806 | Val Loss: 0.2801 | Val Acc: 0.8789
Epoch 9/20
Train Loss: 0.2791 | Val Loss: 0.2825 | Val Acc: 0.8803
Epoch 10/20
Train Loss: 0.2735 | Val Loss: 0.2876 | Val Acc: 0.8783
Early stopping!


C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_binary_model, binary_test_loader)
evaluate(test_true, test_pred, "Pretrained top 10 features on Binary MLP:")

Pretrained top 10 features on Binary MLP: Metrics:
accuracy, precision, recall, f1
0.8809 0.8667 0.9002 0.8831


### Ternary Model

In [ ]:
ternary_train_loader, ternary_val_loader, ternary_test_loader = create_dataloaders(X_trainT['pretrained_top_embeddings'], y_trainT, test_size=0.2, val_size=0.2, batch_size=64)
ternary_model = TernaryMLP(input_dim=len(X_trainT['pretrained_top_embeddings'].iloc[0]))
trained_ternary_model = train_model(ternary_model, ternary_train_loader, ternary_val_loader, num_classes=3)

Epoch 1/20
Train Loss: 0.7968 | Val Loss: 0.6920 | Val Acc: 0.6956
Epoch 2/20
Train Loss: 0.7332 | Val Loss: 0.6587 | Val Acc: 0.7137
Epoch 3/20
Train Loss: 0.7153 | Val Loss: 0.6403 | Val Acc: 0.7250
Epoch 4/20
Train Loss: 0.6999 | Val Loss: 0.6339 | Val Acc: 0.7292
Epoch 5/20
Train Loss: 0.6912 | Val Loss: 0.6303 | Val Acc: 0.7321
Epoch 6/20
Train Loss: 0.6858 | Val Loss: 0.6361 | Val Acc: 0.7328
Epoch 7/20
Train Loss: 0.6778 | Val Loss: 0.6294 | Val Acc: 0.7335
Epoch 8/20
Train Loss: 0.6714 | Val Loss: 0.6244 | Val Acc: 0.7348
Epoch 9/20
Train Loss: 0.6726 | Val Loss: 0.6193 | Val Acc: 0.7340
Epoch 10/20
Train Loss: 0.6644 | Val Loss: 0.6190 | Val Acc: 0.7374
Epoch 11/20
Train Loss: 0.6619 | Val Loss: 0.6217 | Val Acc: 0.7357
Epoch 12/20
Train Loss: 0.6582 | Val Loss: 0.6181 | Val Acc: 0.7380
Epoch 13/20
Train Loss: 0.6586 | Val Loss: 0.6207 | Val Acc: 0.7346
Epoch 14/20
Train Loss: 0.6545 | Val Loss: 0.6224 | Val Acc: 0.7361
Epoch 15/20
Train Loss: 0.6507 | Val Loss: 0.6168 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_ternary_model, ternary_test_loader)
evaluate_multi(test_true, test_pred, "Pretrained top 10 features on ternary MLP:")

Pretrained top 10 features on ternary MLP: Metrics:
accuracy, precision, recall, f1
0.7413 0.7844 0.6513 0.6424


## Custom Word2Vec model features

## Average of Features

#### Binary model

In [ ]:

binary_train_loader, binary_val_loader, binary_test_loader = create_dataloaders(X_trainB['custom_average_embeddings'], y_trainB, test_size=0.2, val_size=0.2, batch_size=64)
binary_model = BinaryMLP(input_dim=len(X_trainB['custom_average_embeddings'].iloc[0]))
trained_binary_model = train_model(binary_model, binary_train_loader, binary_val_loader, num_classes=2)

Epoch 1/20
Train Loss: 0.3617 | Val Loss: 0.2579 | Val Acc: 0.8932
Epoch 2/20
Train Loss: 0.3074 | Val Loss: 0.2470 | Val Acc: 0.8969
Epoch 3/20
Train Loss: 0.2976 | Val Loss: 0.2448 | Val Acc: 0.9010
Epoch 4/20
Train Loss: 0.2925 | Val Loss: 0.2402 | Val Acc: 0.9035
Epoch 5/20
Train Loss: 0.2890 | Val Loss: 0.2377 | Val Acc: 0.9046
Epoch 6/20
Train Loss: 0.2862 | Val Loss: 0.2357 | Val Acc: 0.9032
Epoch 7/20
Train Loss: 0.2815 | Val Loss: 0.2334 | Val Acc: 0.9059
Epoch 8/20
Train Loss: 0.2814 | Val Loss: 0.2378 | Val Acc: 0.8995
Epoch 9/20
Train Loss: 0.2815 | Val Loss: 0.2344 | Val Acc: 0.9059
Epoch 10/20
Train Loss: 0.2784 | Val Loss: 0.2305 | Val Acc: 0.9051
Epoch 11/20
Train Loss: 0.2793 | Val Loss: 0.2317 | Val Acc: 0.9041
Epoch 12/20
Train Loss: 0.2775 | Val Loss: 0.2316 | Val Acc: 0.9069
Epoch 13/20
Train Loss: 0.2770 | Val Loss: 0.2276 | Val Acc: 0.9088
Epoch 14/20
Train Loss: 0.2755 | Val Loss: 0.2304 | Val Acc: 0.9073
Epoch 15/20
Train Loss: 0.2748 | Val Loss: 0.2276 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_binary_model, binary_test_loader)
evaluate(test_true, test_pred, "Custom average features on Binary MLP:")

Custom average features on Binary MLP: Metrics:
accuracy, precision, recall, f1
0.9061 0.9061 0.9061 0.9061


#### Ternary model

In [ ]:
ternary_train_loader, ternary_val_loader, ternary_test_loader = create_dataloaders(X_trainT['custom_average_embeddings'], y_trainT, test_size=0.2, val_size=0.2, batch_size=64)
ternary_model = TernaryMLP(input_dim=len(X_trainT['custom_average_embeddings'].iloc[0]))
trained_ternary_model = train_model(ternary_model, ternary_train_loader, ternary_val_loader, num_classes=3)

Epoch 1/20
Train Loss: 0.7493 | Val Loss: 0.6300 | Val Acc: 0.7320
Epoch 2/20
Train Loss: 0.6935 | Val Loss: 0.6022 | Val Acc: 0.7483
Epoch 3/20
Train Loss: 0.6802 | Val Loss: 0.5990 | Val Acc: 0.7511
Epoch 4/20
Train Loss: 0.6713 | Val Loss: 0.5988 | Val Acc: 0.7513
Epoch 5/20
Train Loss: 0.6677 | Val Loss: 0.5893 | Val Acc: 0.7509
Epoch 6/20
Train Loss: 0.6632 | Val Loss: 0.5862 | Val Acc: 0.7525
Epoch 7/20
Train Loss: 0.6607 | Val Loss: 0.5803 | Val Acc: 0.7542
Epoch 8/20
Train Loss: 0.6598 | Val Loss: 0.5766 | Val Acc: 0.7615
Epoch 9/20
Train Loss: 0.6557 | Val Loss: 0.5810 | Val Acc: 0.7567
Epoch 10/20
Train Loss: 0.6541 | Val Loss: 0.5761 | Val Acc: 0.7572
Epoch 11/20
Train Loss: 0.6513 | Val Loss: 0.5748 | Val Acc: 0.7576
Epoch 12/20
Train Loss: 0.6496 | Val Loss: 0.5779 | Val Acc: 0.7585
Epoch 13/20
Train Loss: 0.6509 | Val Loss: 0.5706 | Val Acc: 0.7630
Epoch 14/20
Train Loss: 0.6506 | Val Loss: 0.5728 | Val Acc: 0.7565
Epoch 15/20
Train Loss: 0.6491 | Val Loss: 0.5727 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_ternary_model, ternary_test_loader)
evaluate_multi(test_true, test_pred, "Custom Average features on ternary MLP:")

Custom Average features on ternary MLP: Metrics:
accuracy, precision, recall, f1
0.7620 0.7661 0.6718 0.6639


## First 10 Features

#### Binary model

In [ ]:

binary_train_loader, binary_val_loader, binary_test_loader = create_dataloaders(X_trainB['custom_top_embeddings'], y_trainB, test_size=0.2, val_size=0.2, batch_size=64)
binary_model = BinaryMLP(input_dim=len(X_trainB['custom_top_embeddings'].iloc[0]))
trained_binary_model = train_model(binary_model, binary_train_loader, binary_val_loader, num_classes=2)

Epoch 1/20
Train Loss: 0.3750 | Val Loss: 0.2823 | Val Acc: 0.8747
Epoch 2/20
Train Loss: 0.3232 | Val Loss: 0.2750 | Val Acc: 0.8802
Epoch 3/20
Train Loss: 0.3113 | Val Loss: 0.2689 | Val Acc: 0.8809
Epoch 4/20
Train Loss: 0.3012 | Val Loss: 0.2673 | Val Acc: 0.8829
Epoch 5/20
Train Loss: 0.2946 | Val Loss: 0.2669 | Val Acc: 0.8834
Epoch 6/20
Train Loss: 0.2884 | Val Loss: 0.2641 | Val Acc: 0.8878
Epoch 7/20
Train Loss: 0.2831 | Val Loss: 0.2640 | Val Acc: 0.8866
Epoch 8/20
Train Loss: 0.2781 | Val Loss: 0.2638 | Val Acc: 0.8868
Epoch 9/20
Train Loss: 0.2708 | Val Loss: 0.2621 | Val Acc: 0.8884
Epoch 10/20
Train Loss: 0.2675 | Val Loss: 0.2605 | Val Acc: 0.8883
Epoch 11/20
Train Loss: 0.2664 | Val Loss: 0.2604 | Val Acc: 0.8886
Epoch 12/20
Train Loss: 0.2626 | Val Loss: 0.2652 | Val Acc: 0.8858
Epoch 13/20
Train Loss: 0.2596 | Val Loss: 0.2660 | Val Acc: 0.8870
Epoch 14/20
Train Loss: 0.2585 | Val Loss: 0.2652 | Val Acc: 0.8884
Early stopping!


C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_binary_model, binary_test_loader)
evaluate(test_true, test_pred, "Custom top 10 features on Binary MLP:")

Custom top 10 features on Binary MLP: Metrics:
accuracy, precision, recall, f1
0.8869 0.9074 0.8618 0.8840


#### Ternary model

In [ ]:
ternary_train_loader, ternary_val_loader, ternary_test_loader = create_dataloaders(X_trainT['custom_top_embeddings'], y_trainT, test_size=0.2, val_size=0.2, batch_size=64)
ternary_model = TernaryMLP(input_dim=len(X_trainT['custom_top_embeddings'].iloc[0]))
trained_ternary_model = train_model(ternary_model, ternary_train_loader, ternary_val_loader, num_classes=3)

Epoch 1/20
Train Loss: 0.7635 | Val Loss: 0.6308 | Val Acc: 0.7286
Epoch 2/20
Train Loss: 0.6992 | Val Loss: 0.6097 | Val Acc: 0.7400
Epoch 3/20
Train Loss: 0.6809 | Val Loss: 0.6086 | Val Acc: 0.7427
Epoch 4/20
Train Loss: 0.6708 | Val Loss: 0.6070 | Val Acc: 0.7435
Epoch 5/20
Train Loss: 0.6633 | Val Loss: 0.6021 | Val Acc: 0.7444
Epoch 6/20
Train Loss: 0.6579 | Val Loss: 0.6013 | Val Acc: 0.7455
Epoch 7/20
Train Loss: 0.6554 | Val Loss: 0.5987 | Val Acc: 0.7461
Epoch 8/20
Train Loss: 0.6499 | Val Loss: 0.6009 | Val Acc: 0.7414
Epoch 9/20
Train Loss: 0.6467 | Val Loss: 0.6018 | Val Acc: 0.7446
Epoch 10/20
Train Loss: 0.6440 | Val Loss: 0.5983 | Val Acc: 0.7460
Epoch 11/20
Train Loss: 0.6419 | Val Loss: 0.6009 | Val Acc: 0.7460
Epoch 12/20
Train Loss: 0.6408 | Val Loss: 0.5977 | Val Acc: 0.7456
Epoch 13/20
Train Loss: 0.6387 | Val Loss: 0.6014 | Val Acc: 0.7458
Epoch 14/20
Train Loss: 0.6361 | Val Loss: 0.5972 | Val Acc: 0.7486
Epoch 15/20
Train Loss: 0.6320 | Val Loss: 0.5976 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_20652\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_ternary_model, ternary_test_loader)
evaluate_multi(test_true, test_pred, "Custom top 10 features on ternary MLP:")

Custom top 10 features on ternary MLP: Metrics:
accuracy, precision, recall, f1
0.7439 0.7460 0.6602 0.6567


## Model Comparisons

In [ ]:
acc = {
    "model_name":["Perceptron","SVM","Average embedding MLP","Top 10 embedding MLP"],
    "Pretrained_embeddings":[0.8458,0.8657,0.8792,0.8809],
    "Custom_embeddings":[0.8548,0.8985,0.9061,0.8869]
}
acc = pd.DataFrame(acc)
acc

,model_name,Pretrained_embeddings,Custom_embeddings
0,Perceptron,0.8458,0.8548
1,SVM,0.8657,0.8985
2,Average embedding MLP,0.8792,0.9061
3,Top 10 embedding MLP,0.8809,0.8869


Custom Embedding Superiority: Custom embeddings consistently outperform pretrained across all models, reinforcing their domain-specific advantage.

MLP Effectiveness: Neural architectures (Average MLP: 90.61%) surpass traditional models (SVM: 89.85%) with custom embeddings, suggesting MLPs better leverage distributed semantic representations.

Aggregation Impact: Average embeddings slightly outperforms Top-10 embeddings, implying comprehensive sequence modeling beats selective token focus for this task.

Model Hierarchy: Performance ascends as Perceptron < SVM < Top-10 MLP < Average MLP, highlighting architectural complexity benefits when paired with quality embeddings.

While custom embeddings universally boost performance, gains scale with model sophistication—MLPs extract 2-3% more accuracy from embeddings than linear models, emphasizing the value of pairing domain-adapted features with sufficiently expressive architectures.

# Convolutional Neural Network

In [ ]:
class BinaryCNN(nn.Module):
    def __init__(self, embedding_dim, sequence_length=50):
        super(BinaryCNN, self).__init__()
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length

        # Convolutional layers
        self.conv1 = nn.Conv1d(1, 50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(50, 10, kernel_size=3, padding=1)

        # Fully connected layer
        self.fc = nn.Linear(10, 1)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        x = x.unsqueeze(1)

        # Convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # Global max pooling
        x = F.max_pool1d(x, x.size(2)).squeeze(2)

        # Dropout and output
        x = self.dropout(x)
        return self.fc(x).squeeze()

class TernaryCNN(nn.Module):
    def __init__(self, embedding_dim, sequence_length=50):
        super(TernaryCNN, self).__init__()
        self.embedding_dim = embedding_dim
        self.sequence_length = sequence_length

        # Convolutional layers
        self.conv1 = nn.Conv1d(1, 50, kernel_size=3, padding=1)
        self.conv2 = nn.Conv1d(50, 10, kernel_size=3, padding=1)

        # Fully connected layer
        self.fc = nn.Linear(10, 3)

        # Dropout
        self.dropout = nn.Dropout(0.5)

    def forward(self, x):

        x = x.unsqueeze(1)

        # Convolutional layers
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))

        # Global max pooling
        x = F.max_pool1d(x, x.size(2)).squeeze(2)

        # Dropout and output
        x = self.dropout(x)
        return self.fc(x)


In [ ]:
def train_cnn(model, train_loader, val_loader, num_classes=2, num_epochs=15, learning_rate=0.001):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = model.to(device)

    # Define loss and optimizer
    criterion = nn.BCEWithLogitsLoss() if num_classes == 2 else nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)

    best_val_acc = 0.0
    patience = 3
    trigger_times = 0

    for epoch in range(num_epochs):
        model.train()
        train_loss = 0.0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            outputs = model(inputs)

            if num_classes == 2:
                loss = criterion(outputs, labels.float())
            else:
                loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()
            train_loss += loss.item() * inputs.size(0)

        train_loss /= len(train_loader.dataset)

        # Validation
        val_loss, val_acc = evaluate_cnn(model, val_loader, num_classes, device)

        print(f'Epoch {epoch+1}/{num_epochs}')
        print(f'Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.4f}')

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            trigger_times = 0
            torch.save(model.state_dict(), 'best_cnn_model.pth')
        else:
            trigger_times += 1
            if trigger_times >= patience:
                print("Early stopping!")
                break

    model.load_state_dict(torch.load('best_cnn_model.pth'))
    return model

In [ ]:
def evaluate_cnn(model, data_loader, num_classes, device='cpu'):
    model.eval()
    total_loss = 0.0
    correct = 0
    total = 0
    criterion = nn.BCEWithLogitsLoss() if num_classes == 2 else nn.CrossEntropyLoss()

    with torch.no_grad():
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            if num_classes == 2:
                loss = criterion(outputs, labels.float())
                preds = (torch.sigmoid(outputs) > 0.5).long()
            else:
                loss = criterion(outputs, labels)
                preds = torch.argmax(outputs, dim=1)

            total_loss += loss.item() * inputs.size(0)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    avg_loss = total_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


In [ ]:
def get_predictions(model, dataloader, device='cpu'):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs = inputs.to(device)
            outputs = model(inputs)

            preds = (torch.sigmoid(outputs) > 0.5).cpu().numpy()

            all_preds.extend(preds)
            all_labels.extend(labels.numpy())

    return np.array(all_labels), np.array(all_preds)

## Pretrained Word2Vec features

### Binary model

In [ ]:
binary_train_loader, binary_val_loader, binary_test_loader = create_dataloaders(X_trainB["pretrained_average_50_embeddings"], y_trainB, test_size=0.2, val_size=0.2, batch_size=64)
binary_model = BinaryCNN(embedding_dim=len(X_trainB['pretrained_average_50_embeddings'].iloc[0]))
trained_binary_model = train_model(binary_model, binary_train_loader, binary_val_loader, num_classes=2)

Epoch 1/20
Train Loss: 0.6936 | Val Loss: 0.6925 | Val Acc: 0.5221
Epoch 2/20
Train Loss: 0.6900 | Val Loss: 0.6852 | Val Acc: 0.5451
Epoch 3/20
Train Loss: 0.6874 | Val Loss: 0.6855 | Val Acc: 0.5447
Epoch 4/20
Train Loss: 0.6877 | Val Loss: 0.6844 | Val Acc: 0.5477
Epoch 5/20
Train Loss: 0.6853 | Val Loss: 0.6832 | Val Acc: 0.5428
Epoch 6/20
Train Loss: 0.6840 | Val Loss: 0.6803 | Val Acc: 0.5514
Epoch 7/20
Train Loss: 0.6836 | Val Loss: 0.6804 | Val Acc: 0.5507
Epoch 8/20
Train Loss: 0.6836 | Val Loss: 0.6784 | Val Acc: 0.5568
Epoch 9/20
Train Loss: 0.6819 | Val Loss: 0.6763 | Val Acc: 0.5747
Epoch 10/20
Train Loss: 0.6792 | Val Loss: 0.6716 | Val Acc: 0.5856
Epoch 11/20
Train Loss: 0.6726 | Val Loss: 0.6614 | Val Acc: 0.6157
Epoch 12/20
Train Loss: 0.6690 | Val Loss: 0.6584 | Val Acc: 0.6158
Epoch 13/20
Train Loss: 0.6668 | Val Loss: 0.6547 | Val Acc: 0.6241
Epoch 14/20
Train Loss: 0.6660 | Val Loss: 0.6576 | Val Acc: 0.6116
Epoch 15/20
Train Loss: 0.6657 | Val Loss: 0.6535 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_binary_model, binary_test_loader)
evaluate(test_true, test_pred, "Pretrained average of 50 features on Binary CNN:")

Pretrained average of 50 features on Binary CNN: Metrics:
accuracy, precision, recall, f1
0.6322 0.6797 0.4998 0.5760


### Ternary model

In [ ]:
ternary_train_loader, ternary_val_loader, ternary_test_loader = create_dataloaders(X_trainT['pretrained_average_50_embeddings'], y_trainT, test_size=0.2, val_size=0.2, batch_size=64)
ternary_model = TernaryCNN(embedding_dim=len(X_trainT['pretrained_average_50_embeddings'].iloc[0]))
trained_ternary_model = train_model(ternary_model, ternary_train_loader, ternary_val_loader, num_classes=3)

Epoch 1/20
Train Loss: 1.0587 | Val Loss: 1.0516 | Val Acc: 0.4542
Epoch 2/20
Train Loss: 1.0482 | Val Loss: 1.0395 | Val Acc: 0.4683
Epoch 3/20
Train Loss: 1.0399 | Val Loss: 1.0317 | Val Acc: 0.4817
Epoch 4/20
Train Loss: 1.0368 | Val Loss: 1.0293 | Val Acc: 0.4875
Epoch 5/20
Train Loss: 1.0352 | Val Loss: 1.0296 | Val Acc: 0.4817
Epoch 6/20
Train Loss: 1.0337 | Val Loss: 1.0263 | Val Acc: 0.4857
Epoch 7/20
Train Loss: 1.0334 | Val Loss: 1.0238 | Val Acc: 0.4948
Epoch 8/20
Train Loss: 1.0318 | Val Loss: 1.0262 | Val Acc: 0.4853
Epoch 9/20
Train Loss: 1.0308 | Val Loss: 1.0233 | Val Acc: 0.4955
Epoch 10/20
Train Loss: 1.0309 | Val Loss: 1.0211 | Val Acc: 0.4974
Epoch 11/20
Train Loss: 1.0303 | Val Loss: 1.0189 | Val Acc: 0.5038
Epoch 12/20
Train Loss: 1.0293 | Val Loss: 1.0166 | Val Acc: 0.5061
Epoch 13/20
Train Loss: 1.0283 | Val Loss: 1.0180 | Val Acc: 0.5034
Epoch 14/20
Train Loss: 1.0285 | Val Loss: 1.0155 | Val Acc: 0.5064
Epoch 15/20
Train Loss: 1.0265 | Val Loss: 1.0145 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
print("ternary CNN on pretrained embeddings accuracy:")
evaluate_cnn(ternary_model, ternary_test_loader, 3)[1]

ternary CNN on pretrained embeddings accuracy:


0.511425

## Custom Word2Vec features

### Binary model

In [ ]:

binary_train_loader, binary_val_loader, binary_test_loader = create_dataloaders(X_trainB['pretrained_average_50_embeddings'], y_trainB, test_size=0.2, val_size=0.2, batch_size=64)
binary_model = BinaryCNN(embedding_dim=len(X_trainB['pretrained_average_50_embeddings'].iloc[0]))
trained_binary_model = train_model(binary_model, binary_train_loader, binary_val_loader, num_classes=2)

Epoch 1/20
Train Loss: 0.6938 | Val Loss: 0.6927 | Val Acc: 0.5725
Epoch 2/20
Train Loss: 0.6887 | Val Loss: 0.6762 | Val Acc: 0.6108
Epoch 3/20
Train Loss: 0.6776 | Val Loss: 0.6693 | Val Acc: 0.6237
Epoch 4/20
Train Loss: 0.6737 | Val Loss: 0.6631 | Val Acc: 0.6320
Epoch 5/20
Train Loss: 0.6716 | Val Loss: 0.6586 | Val Acc: 0.6272
Epoch 6/20
Train Loss: 0.6675 | Val Loss: 0.6480 | Val Acc: 0.6456
Epoch 7/20
Train Loss: 0.6662 | Val Loss: 0.6509 | Val Acc: 0.6395
Epoch 8/20
Train Loss: 0.6632 | Val Loss: 0.6462 | Val Acc: 0.6474
Epoch 9/20
Train Loss: 0.6652 | Val Loss: 0.6522 | Val Acc: 0.6388
Epoch 10/20
Train Loss: 0.6640 | Val Loss: 0.6447 | Val Acc: 0.6518
Epoch 11/20
Train Loss: 0.6612 | Val Loss: 0.6463 | Val Acc: 0.6516
Epoch 12/20
Train Loss: 0.6602 | Val Loss: 0.6469 | Val Acc: 0.6487
Epoch 13/20
Train Loss: 0.6585 | Val Loss: 0.6390 | Val Acc: 0.6567
Epoch 14/20
Train Loss: 0.6530 | Val Loss: 0.6329 | Val Acc: 0.6616
Epoch 15/20
Train Loss: 0.6499 | Val Loss: 0.6303 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
test_true, test_pred = get_predictions(trained_binary_model, binary_test_loader)
evaluate(test_true, test_pred, "Custom average of 50 features on ternary CNN:")

Custom average of 50 features on ternary CNN: Metrics:
accuracy, precision, recall, f1
0.6640 0.6561 0.6891 0.6722


### Ternary model

In [ ]:

ternary_train_loader, ternary_val_loader, ternary_test_loader = create_dataloaders(X_trainT['pretrained_average_50_embeddings'], y_trainT, test_size=0.2, val_size=0.2, batch_size=64)
ternary_model = TernaryCNN(embedding_dim=len(X_trainT['pretrained_average_50_embeddings'].iloc[0]))
trained_ternary_model = train_model(ternary_model, ternary_train_loader, ternary_val_loader, num_classes=3)

Epoch 1/20
Train Loss: 1.0587 | Val Loss: 1.0521 | Val Acc: 0.4835
Epoch 2/20
Train Loss: 1.0450 | Val Loss: 1.0327 | Val Acc: 0.4892
Epoch 3/20
Train Loss: 1.0385 | Val Loss: 1.0311 | Val Acc: 0.4809
Epoch 4/20
Train Loss: 1.0355 | Val Loss: 1.0228 | Val Acc: 0.5004
Epoch 5/20
Train Loss: 1.0310 | Val Loss: 1.0197 | Val Acc: 0.4996
Epoch 6/20
Train Loss: 1.0289 | Val Loss: 1.0138 | Val Acc: 0.5107
Epoch 7/20
Train Loss: 1.0279 | Val Loss: 1.0125 | Val Acc: 0.5126
Epoch 8/20
Train Loss: 1.0263 | Val Loss: 1.0120 | Val Acc: 0.5128
Epoch 9/20
Train Loss: 1.0262 | Val Loss: 1.0171 | Val Acc: 0.5064
Epoch 10/20
Train Loss: 1.0254 | Val Loss: 1.0089 | Val Acc: 0.5147
Epoch 11/20
Train Loss: 1.0230 | Val Loss: 1.0090 | Val Acc: 0.5130
Epoch 12/20
Train Loss: 1.0214 | Val Loss: 1.0051 | Val Acc: 0.5194
Epoch 13/20
Train Loss: 1.0198 | Val Loss: 1.0046 | Val Acc: 0.5164
Epoch 14/20
Train Loss: 1.0177 | Val Loss: 1.0026 | Val Acc: 0.5205
Epoch 15/20
Train Loss: 1.0168 | Val Loss: 0.9988 | Val A

C:\Users\Aditya Roy\AppData\Local\Temp\ipykernel_10004\3495807950.py:55: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [ ]:
# test_true, test_pred = get_predictions(trained_ternary_model, ternary_test_loader)
print("ternary CNN on pretrained embeddings accuracy:")
evaluate_cnn(ternary_model, ternary_test_loader, 3)[1]

ternary CNN on pretrained embeddings accuracy:


0.5253